# Transversal Gates

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/application/qecc/transversal.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Transversal gates are important in fault-tolerant quantum computing, as they allow for operations that can be performed on multiple qubits simultaneously without introducing errors that propagate through the system. These gates are particularly useful in stabilizer codes and other error-correcting codes. This tutorial will cover the basics of transversal gates and their properties, especially reproduce results in the arxiv paper [arxiv-link](https://arxiv.org/abs/2504.20847).

In [ ]:
import functools
import numpy as np

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi

np_rng = np.random.default_rng()
hf_kron = lambda *x: functools.reduce(np.kron, x) #tensor product of matrices

## Stabilizer codes

Let's begin with two famous stabilizer codes: the 5-qubit code and the Steane code.

### 5-qubit code ((5,2,3))

A quantum error-correcting code (QECC) is a subspace specified by a set of logical basis states

In [ ]:
code,info = numqi.qec.get_code_subspace('523')

# logical states are some special state vector in the complete Hilbert space
print('logical 0:', (code[0]*4).astype(np.int64), sep='\n')
print('logical 1:', (code[1]*4).astype(np.int64), sep='\n')

Via basis matrix multiplication, one can verify that logical X/Y/Z gate can be implemented by applying the Pauli gates on all physical qubits. Such gates are called transversal gates.

In [ ]:
# logical X = X * X * X * X * X
logicalX = code.conj() @ hf_kron(*[-numqi.gate.X]*5) @ code.T
print('logical X:', logicalX, sep='\n')

# logical Y = Y * Y * Y * Y * Y
logicalY = code.conj() @ hf_kron(*[-numqi.gate.Y]*5) @ code.T
print('\nlogical Y:', logicalY, sep='\n')

# logical Z = Z * Z * Z * Z * Z
logicalZ = code.conj() @ hf_kron(*[numqi.gate.Z]*5) @ code.T
print('\nlogical Z:', logicalZ, sep='\n')

Besides transversal logical X/Y/Z gates, the 5-qubit code also has transversal logical $F=HS^\dagger$ gate (in Bloch sphere, $F$ gate is a rotation by 120 degrees around (1,1,1) axis). Although 5-qubit code has transversal logical $F=HS^\dagger$, but it does not have transversal logical $H$ and $S$ gates. To support transversal logical $H$ and $S$ gates, we need to use the Steane code which will be introduced in the next section.

In [ ]:
op_physical = numqi.gate.Y @ numqi.gate.H @ numqi.gate.S.conj()
logicalF = -np.exp(1j*np.pi/4)*code.conj() @ hf_kron(*[op_physical]*5) @ code.T
print('logical F:', logicalF, sep='\n')

Besides transversal gates, `numqi` also calculate Shor's weight enumerator of the code. Code distance can be read from the weight enumerator which is the first non-equal term in the weight enumerator with its dual. For the 5-qubit code, the distance is 3, which means `A3,B3` are different.

In [ ]:
print('Shor weight enumerator (A0,A1,A2,A3,A4,A5):', info['qweA'], sep='\n')
print('\ndual of Shor weight enumerator (B0,B1,B2,B3,B4,B5):', info['qweB'], sep='\n')

### Steane code ((7,2,3))

In [ ]:
code,info = numqi.qec.get_code_subspace('steane')

print('code.shape:', code.shape)
print('stabilizer:', info['stab'])
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

Steane code support transversal logical $H$ and $S$ gates, as demonstrated below.

TASK: given these two transversal logical gates, can you find the implementation of transversal logical gate $F=HS^\dagger$?

In [ ]:
# logical H = H * H * H * H * H * H * H
logicalH = code.conj() @ hf_kron(*[numqi.gate.H]*7) @ code.T
print('logical H:', logicalH, sep='\n')

# logical S = (S * S * S * S * S * S * S)^\dagger
logicalS = code.conj() @ hf_kron(*[numqi.gate.S.conj()]*7) @ code.T
print('\nlogical S:', logicalS, sep='\n')

Actually, transversal logical gates make a group: given two transversal logical gates, then their product is also a transversal logical gate.

Furthermore, Eastin-Knill theorem states that any transversal group of nontrivial (distance >1) QECC is a finite subgroup of SU(K) where K is the dimension of the logical subspace.

Finite subgroup of SU(2) has been classified as follows: cyclic groups $C_{2m}$, binary dihedral groups $\mathrm{BD}_{2m}$, and the three exceptional groups: binary tetrahedral group (2T), binary octahedral group (2O), and binary icosahedral group (2I).

| group | notable elements | generators | order |
| :-: | :-: | :-: | :-: |
| $C_{2m}$ | $Z(2\pi/m)$ | $Z(2\pi/m)$ | 2m |
| $\mathrm{BD}_{2m}$ | $\hat{X},Z(2\pi/m)$ | $\hat{X},Z(2\pi/m)$ | 4m |
| 2T | $\hat{X},\hat{Z},F$ | $\hat{X},F$ | 24 |
| 2O (Clifford) | $\hat{X},\hat{Z},\hat{S},\hat{H},F$ | $\hat{S},\hat{H}$ | 48 |
| 2I | $\hat{X},\hat{Z},F,\Phi$ | $\hat{X},\hat{Z}\Phi$ | 120 |

According to classification, the transversal group of the 5-qubit code is 2T, and the transversal group of the Steane code is 2O which is also isomorphic to 1-qubit Clifford group.

Transversal group of stabilizer code is quite limited, below we demonstrate that transversal group of non-stabilizer code is much richer, which is the main result of our paper [arxiv-link](https://arxiv.org/abs/2504.20847).


## Non-stabilizer code ((6,2,3))

### C10

In [ ]:
code,info = numqi.qec.q623.get_C10(return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])


`info['su2']` stores the SU(2) gates applied to each qubits that implement a Z-rotation logical gate $Z(2\pi/5)$.

In [ ]:
print('physical gates for each qubit:')
for i0,op in enumerate(info['su2']):
    print(f'qubit {i0}:', op, sep='\n')

op_logical = code.conj() @ hf_kron(*info['su2']) @ code.T
print('\nlogical Z(2pi/5):', op_logical, sep='\n')

### SO(5) code

when `vece` has at least 4 nonzero entries, then the corresponding code has no transversal logical gates except trivial identity, `C2` group.

In [ ]:
tmp0 = np_rng.normal(size=4)
vece = tmp0 / np.linalg.norm(tmp0)
code = numqi.qec.q623.get_SO5_code_with_transversal_gate(vece) #no trasversal gate

when 5-dimensional `vece` has at most 3 nonzero entries, then transversal group is `C4`.

In [ ]:
tmp0 = np_rng.normal(size=3)
vece = tmp0 / np.linalg.norm(tmp0)
code,info = numqi.qec.q623.get_SO5_code_with_transversal_gate(vece) #no trasversal gate

print("physical gates' shape:", info['su2'].shape)
logicalZ = code.conj() @ hf_kron(*info['su2']) @ code.T
print('logical Z:', logicalZ, sep='\n')

when `vece` has at most 2 nonzero entries, then transversal group is `BD4`.

In [ ]:
tmp0 = np_rng.normal(size=2)
vece = tmp0 / np.linalg.norm(tmp0)
code,info = numqi.qec.q623.get_SO5_code_with_transversal_gate(vece) #no trasversal gate

logicalZ = code.conj() @ hf_kron(*info['su2Z']) @ code.T
print('logical Z:', np.around(logicalZ,10), sep='\n')

logicalX = code.conj() @ hf_kron(*info['su2X']) @ code.T
print('logical X:', np.around(logicalX,10), sep='\n')

### ((6,2,3)) from ((5,2,3)) stabilizer code

see appendix of [arxiv-link](https://arxiv.org/abs/2504.20847) for details.

## Non-stabilizer code ((7,2,3))

### 2T, Cyclic code

[arxiv-link](https://arxiv.org/abs/2410.07983) Characterizing Quantum Codes via the Coefficients in Knill-Laflamme Conditions

Parametrized with the signature norm $\lambda^*\in[0,\sqrt{7}]$, when $\lambda^*=0$, it becomes the Steane code, and when $\lambda^*=\sqrt{7}$, it becomes a permutational-invariant code.

In [ ]:
# 0<lambda<sqrt(7), Cyclic code, 2T
code,info = numqi.qec.q723.get_cyclic_code(lambda2=np_rng.uniform(0,7), sign='++', return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

In [ ]:
logicalX = code.conj() @ hf_kron(*[numqi.gate.X]*7) @ code.T
print('\nlogical X:', logicalX, sep='\n')

logicalF = code.conj() @ hf_kron(*[numqi.gate.H @ numqi.gate.S.conj()]*7) @ code.T
print('\nlogical F:', logicalF, sep='\n')

When $\lambda^*=0$, extra transversal logical gates are $H$ and $S$, which is the same as Steane code.

In [ ]:
code,info = numqi.qec.q723.get_cyclic_code(lambda2=0, sign='++', return_info=True)

logicalH = code.conj() @ hf_kron(*[numqi.gate.H]*7) @ code.T
print('logical H:', logicalH, sep='\n')

logicalS = code.conj() @ hf_kron(*[numqi.gate.S.conj()]*7) @ code.T
print('\nlogical S:', logicalS, sep='\n')

When $\lambda^*=\sqrt{7}$, extra transversal logical gate is $\Phi$.

In [ ]:
code,info = numqi.qec.q723.get_cyclic_code(lambda2=7, sign='++', return_info=True)

physical_op = numqi.qec.su2_finite_subgroup_gate_dict['Phi']
logicalPhi = code.conj() @ hf_kron(*[physical_op]*7) @ code.T
print('logical Psi:', logicalPhi, sep='\n')

### 2I, permutation-invariant code

In [ ]:
coeff = np.zeros((2,8), dtype=np.complex128)
coeff[[0,0,1,1],[0,5,2,7]] = np.array([np.sqrt(3), np.sqrt(7)*1j, np.sqrt(7)*1j, np.sqrt(3)]) / np.sqrt(10)
code = coeff @ (numqi.dicke.get_dicke_basis(7, 2)[::-1])

qweA,qweB = numqi.qec.get_weight_enumerator(code)
print('Shor weight enumerator:', np.around(qweA,3))
print('dual of Shor weight enumerator:', np.around(qweB,3))


In [ ]:
logicalX = code.conj() @ hf_kron(*[numqi.gate.X]*7) @ code.T
print('logical X:', logicalX, sep='\n')

logicalZ5 = code.conj() @ hf_kron(*[numqi.gate.rz(6*np.pi/5)]*7) @ code.T
print('\nlogical Z(2pi/5):', logicalZ5, sep='\n')

hfR = lambda a,b,t=1: numqi.gate.I*np.cos(t*np.pi/5) + 1j*np.sin(t*np.pi/5)/np.sqrt(5) * (a*numqi.gate.Y + b*numqi.gate.Z)
physical_op = hfR(-2,-1,3)
logicalR = code.conj() @ hf_kron(*[physical_op]*7) @ code.T #hfR(-2,1)
print('\nlogical R(-2,1):', logicalR, sep='\n')


### 2I, lambda*=0

In [ ]:
code,info = numqi.qec.q723.get_2I_lambda0(theta=np_rng.uniform(0,2*np.pi),
                phi=np_rng.uniform(0,2*np.pi), sign='+', return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

In [ ]:
logicalX = code.conj() @ hf_kron(*[numqi.gate.X]*7) @ code.T
print('logical X:', logicalX, sep='\n')

logicalZ5 = code.conj() @ hf_kron(*info['su2']) @ code.T
print('\nlogical Z(2pi/5):', logicalZ5, sep='\n')

# hfR = lambda a,b,t=1: numqi.gate.I*np.cos(t*np.pi/5) + 1j*np.sin(t*np.pi/5)/np.sqrt(5) * (a*numqi.gate.Y + b*numqi.gate.Z)
# physical_op = hfR(-2,-1,3)
logicalR = code.conj() @ hf_kron(*info['su2R']) @ code.T #hfR(-2,1)
print('\nlogical R(-2,1):', logicalR, sep='\n')


### 2I, lambda*=0.75

In [ ]:
code,info = numqi.qec.q723.get_2I_lambda075(np_rng.uniform(0,np.sqrt(5/16)),
                        sign=np.array([1,1,1]), return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

In [ ]:
logicalX = code.conj() @ hf_kron(*[numqi.gate.X]*7) @ code.T
print('logical X:', logicalX, sep='\n')

logicalZ5 = code.conj() @ hf_kron(*info['su2']) @ code.T
print('\nlogical Z(2pi/5):', logicalZ5, sep='\n')

# hfR = lambda a,b,t=1: numqi.gate.I*np.cos(t*np.pi/5) + 1j*np.sin(t*np.pi/5)/np.sqrt(5) * (a*numqi.gate.Y + b*numqi.gate.Z)
# physical_op = hfR(-2,-1,3)
logicalR = code.conj() @ hf_kron(*info['su2R']) @ code.T #hfR(-2,1)
print('\nlogical R(-2,1):', logicalR, sep='\n')


### 2O lambda*=2

In [ ]:
code,info = numqi.qec.q723.get_2O_X5(return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

In [ ]:
logicalX = code.conj() @ hf_kron(*info['su2X']) @ code.T
print('logical X:', logicalX, sep='\n')

logicalYSY = code.conj() @ hf_kron(*info['su2YSY']) @ code.T
print('\nlogical Y(pi/4)SY(-pi/4):', logicalYSY, sep='\n')


### BD16, transversal T

In [ ]:
theta = np_rng.uniform(0,2*np.pi,size=2)
sign = np_rng.integers(2, size=7)*2 - 1
code,info = numqi.qec.q723.get_BD16_veca1222233(theta[0], theta[1], sign=sign, return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

In [ ]:
logicalX = code.conj() @ hf_kron(*[numqi.gate.X]*7) @ code.T
print('logical X:', logicalX, sep='\n')

logicalT = code.conj() @ hf_kron(*info['su2']) @ code.T
print('\nlogical T:', logicalT, sep='\n')


### BD32, transversal sqrt(T)

In [ ]:
sign = np_rng.integers(2, size=9)*2 - 1
code, info = numqi.qec.q723.get_BD32(np_rng.uniform(0,np.sqrt(1/8)), sign=sign, return_info=True)

print('code.shape:', code.shape)
print('Shor weight enumerator:', info['qweA'])
print('dual of Shor weight enumerator:', info['qweB'])

In [ ]:
logicalX = code.conj() @ hf_kron(*[numqi.gate.X]*7) @ code.T
print('logical X:', logicalX, sep='\n')

logicalSqrtT = code.conj() @ hf_kron(*info['su2']) @ code.T
print('\nlogical Sqrt(T):', logicalSqrtT, sep='\n')


### More QECCs

Here we provides a list of available QECCs in `numqi.qec`:

In [ ]:
tmp0 = [x for x in dir(numqi.qec.q723) if x.startswith('get_')]
for x in tmp0:
    print(f'numqi.qec.q723.{x}()')

In [ ]:
tmp0 = [x for x in dir(numqi.qec.q823) if x.startswith('get_')]
for x in tmp0:
    print(f'numqi.qec.q823.{x}()')